# 📊 YouTube Trend Analytics — Script Generation Notebook

This notebook prepares all essential Python scripts required for the **Daily YouTube Trend Automation System**, including:

- Fetching supported country codes
- Fetching YouTube category mappings
- Utility function for converting video durations
- Core script for fetching trending data for any region
- Main automation script used in GitHub Actions

All files generated from this notebook will be committed to the GitHub repository and used by the daily automation workflow.

---


## 🌍 Fetch All Supported YouTube Countries

YouTube supports trending videos for many regions.  
This cell:
- Loads the YouTube Data API key from the environment
- Calls the `i18nRegions` endpoint
- Extracts the list of region codes (IN, US, GB, BR, etc.)
- Saves them into `countries.json`

These region codes are used later for fetching trending videos for each country.


In [1]:
from googleapiclient.discovery import build
from dotenv import load_dotenv
import os
import json

# Load environment variables (YOUTUBE_API_KEY)
load_dotenv()

# Initialize YouTube Data API client
API_KEY = os.getenv('YOUTUBE_API_KEY')
YOUTUBE = build("youtube", "v3", developerKey=API_KEY)

def get_all_countries():
    """
    Fetch all supported YouTube region codes (e.g., IN, US, BR, GB...).
    Returns a list of country codes.
    """
    request = YOUTUBE.i18nRegions().list(part="snippet")
    response = request.execute()

    countries = []
    for item in response["items"]:
        countries.append(item["id"])
    return countries

# Fetch countries and save to JSON file
countries = get_all_countries()
with open("../countries.json", "w") as f:
    json.dump(countries, f, indent=4)

In [2]:
# Print first 5 elements of the countries list
countries[:5]

['AE', 'BH', 'DZ', 'EG', 'IQ']

## 🗂️ Generate Category Mapping File

This cell:
- Fetches video categories for each country
- Merges each one into a single dictionary
- Saves the category ID → category name mapping into `categories.json`

This file is used by all future scripts to translate YouTube's category IDs to readable names.


In [3]:
def get_category_map():
    """
    Fetch YouTube video categories for each country, merge them and return a mapping:
        {category_id: category_name}
    """
    categories = {} 
    for country in countries:
        request = YOUTUBE.videoCategories().list(
            part="snippet",
            regionCode=country
        )
        response = request.execute()
        
        for item in response["items"]:
            if item["id"] not in categories:
                categories[item["id"]] = item["snippet"]["title"]
    return categories

# Fetch categories and save to JSON file
categories = get_category_map()
with open("../categories.json", "w") as f:
    json.dump(categories, f, indent=4)

In [6]:
# Print categories dictionary
categories

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers',
 '29': 'Nonprofits & Activism'}

## ⏱️ Create Duration Conversion Utility

This script (`convert_duration.py`) contains a helper function that converts
ISO-8601 duration strings such as:

- PT5M20S  
- PT1H2M5S  
- PT30S  

into total seconds.

This utility will be imported inside the trending fetch script.


In [26]:
%%writefile ../scripts/convert_duration.py
import re

def duration_to_seconds(duration):
    """
    Convert ISO-8601 YouTube duration strings into total seconds.

    Examples:
        PT5M20S → 320 seconds
        PT1H2M5S → 3725 seconds
        PT30S → 30 seconds

    Parameters:
        duration (str): ISO-8601 duration string

    Returns:
        int: Duration in seconds
    """
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', duration)
    if not match:
        return 0
    
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0
    
    return hours * 3600 + minutes * 60 + seconds

Overwriting ../scripts/convert_duration.py


## 📥 Create Script to Fetch Trending Videos

This script (`fetch_youtube_trend.py`) defines the function `get_trending_videos(region)`:

- Calls the YouTube API for a specific region
- Retrieves snippet, statistics, and content details
- Maps video category IDs → category names using `categories.json`
- Converts video duration into seconds
- Returns a clean pandas DataFrame

This script is used by the main automation workflow.


In [1]:
%%writefile ../scripts/fetch_youtube_trend.py
from googleapiclient.discovery import build
import pandas as pd
from dotenv import load_dotenv
import json
import os
from datetime import datetime, timezone
from convert_duration import duration_to_seconds

# Load API Key
load_dotenv()

API_KEY = os.getenv('YOUTUBE_API_KEY')
YOUTUBE = build("youtube", "v3", developerKey=API_KEY)

# Load category mapping
with open("../categories.json", "r") as f:
    cat_map = json.load(f)

def get_trending_videos(region):
    """
    Fetch top 50 trending videos for a given region.

    Parameters:
        region (str): Country code (e.g., 'IN', 'US')

    Returns:
        pd.DataFrame: Cleaned table of trending videos and metadata
    """
    request = YOUTUBE.videos().list(
        part="snippet,statistics,contentDetails",
        chart="mostPopular",
        regionCode=region,
        maxResults=50
    )
    response = request.execute()

    rows = []
    for v in response.get("items", []):
        snippet = v["snippet"]
        stats = v.get("statistics", {})
        content = v.get("contentDetails", {})
        status = v.get("status", {})
        cid = snippet.get("categoryId")

        rows.append({
            "video_id": v["id"],
            "country": region,
            "fetched_at": datetime.now(timezone.utc).isoformat(),

            # Snippet
            "published_at": snippet.get("publishedAt"),
            "title": snippet.get("title"),
            "localized_title": snippet.get("localized", {}).get("title"),
            "channel_title": snippet.get("channelTitle"),
            "channel_id": snippet.get("channelId"),
            "category_id": cid,
            "category_name": cat_map.get(cid, "Unknown"),
            "tags": ", ".join(snippet.get("tags", [])),
            "tag_count": len(snippet.get("tags", [])),
            "thumbnail": snippet.get("thumbnails", {}).get("high", {}).get("url"),
            "default_language": snippet.get("defaultLanguage"),
            "audio_language": snippet.get("defaultAudioLanguage"),
            "is_live": snippet.get("liveBroadcastContent") == "live",

            # Content details
            "duration": duration_to_seconds(content.get("duration")),
            "duration_raw": content.get("duration"),
            "definition": content.get("definition"),
            "caption_available": content.get("caption") == "true",
            "licensed_content": content.get("licensedContent", False),
            "embeddable": status.get("embeddable", False),
            "made_for_kids": status.get("madeForKids", False),

            # Stats
            "views": int(stats.get("viewCount", 0)),
            "likes": int(stats.get("likeCount", 0)),
            "comments": int(stats.get("commentCount", 0)),
        })


    df = pd.DataFrame(rows)
    df['published_at'] = pd.to_datetime(df['published_at'])
    df['fetched_at'] = pd.to_datetime(df['fetched_at'])

    return df

Overwriting ../scripts/fetch_youtube_trend.py


## 🚀 Create Main Automation Script

This script (`main.py`) orchestrates the full workflow:

- Loads the list of countries from `countries.json`
- Creates folder structure: `data/<year>/<country>/...`
- Fetches trending videos for each region
- Saves Country-level daily CSV (`trending_IN_2025-12-07.csv`)

This script is the one executed daily by GitHub Actions.


In [1]:
%%writefile ../scripts/main.py
import pandas as pd
import datetime
import os
import time
import json
from fetch_youtube_trend import get_trending_videos

# Load list of YouTube-supported countries
with open("../countries.json", 'r') as f:
    countries = json.load(f)

def run():
    """
    Main workflow:
    - Create folder structure for the current year
    - Fetch trending videos for every country
    - Save country-level CSV files
    """
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    year = today.split("-")[0]
    month = today.split("-")[1]
    BASE_DIR = os.path.abspath(".")
    DATA_DIR = os.path.join(BASE_DIR, "..", "data")
    
    for country in countries:
        print(f"\nFetching trending videos for {country}...")

        try:
            df = get_trending_videos(country)

            # Skip empty results
            if df.empty:
                print(f"No trending data for {country}, skipping.")
                continue

            # Folder for the specific country and the date
            COUNTRY_DIR = os.path.join(DATA_DIR, f"country={country}", f"year={year}", f"month={month}")
            os.makedirs(COUNTRY_DIR, exist_ok=True)

            # Save daily file
            file_path = os.path.join(
                COUNTRY_DIR,
                f"trending_{country}_{today}.csv"
            )

            df.to_csv(file_path, index=False)
            print(f"Saved → {file_path}")

            time.sleep(0.3)  # Avoid API quota bursts

        except Exception as e:
            print(f"Error fetching {country}: {e}")

# Execute workflow when script is run
if __name__ == "__main__":
    run()

Overwriting ../scripts/main.py
